In [ ]:
import requests
import json


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# Set up the Chrome driver
driver = webdriver.Chrome()

# Load the URL in the driver
driver.get("https://www.brickeconomy.com/sitemap/")

# Find the div element with class 'theme'

theme_divs = driver.find_elements(by=By.CLASS_NAME, value='theme')
for theme_div in theme_divs:
    # Find all the a elements inside the div element
    a_elements = theme_div.find_elements(by=By.TAG_NAME, value='a')
    a_s =[]
    # Print the href attribute of each a element
    for a in a_elements:
        print(a.get_attribute('href'))
        a_s.append(a.get_attribute('href'))

# Quit the driver
driver.quit()



In [ ]:
# read file 'lego themes'
with open('lego_themes', 'r') as f:
    lego_themes = f.readlines()


In [ ]:
# for all the urls, get all the sets
for theme in lego_themes:
    driver = webdriver.Chrome()

    # Load the URL in the driver
    driver.get(theme)

    headings = driver.find_elements(by=By.TAG_NAME, value='h4')
    # each h4 contains an a element
    for heading in headings:
        a_elements = heading.find_elements(by=By.TAG_NAME, value='a')
        a_s =[]
        # Print the text of each a element
        for a in a_elements:
            print(a.text)
            a_s.append(a.text)

        # Quit the driver
    driver.quit()

In [3]:
# read file 'set_ids'
with open('set_ids', 'r') as f:
    set_ids = f.readlines()
# split by " " into a tuple
set_ids = [set_id.split(" ") for set_id in set_ids]
# reorder as (id, name), where te first element is the id and all the rest is the name
set_ids = [(set_id[0], " ".join(set_id[1:])) for set_id in set_ids]
set_ids = [(set_id[0], set_id[1].replace("\n", "")) for set_id in set_ids]
# remove all set_ids which contain alphabets in the 0th index
set_ids = [set_id for set_id in set_ids if not set_id[0].isalpha()]

In [ ]:
set_ids

In [ ]:
from selenium import webdriver

def get_set_cost(set_num):
    # URL for set page on Brickeconomy
    url = f"https://www.brickeconomy.com/set/{set_num}-1/"
    
    # Set up the Chrome driver
    driver = webdriver.Chrome()
    
    # Load the URL in the driver
    driver.get(url)
    
    # Find the element containing the retail price
    try: retail_price_element = driver.find_element_by_xpath("//div[contains(text(), 'Retail price')]")
    except: return None, None
    # Get the price from the next sibling element
    price_str = retail_price_element.find_element_by_xpath("./following-sibling::div").text.strip().replace('$', '').replace(',', '')
    price = float(price_str)

    value = driver.find_element_by_xpath("//div[contains(text(), 'Value')]")
    value_str = value.find_element_by_xpath("./following-sibling::div").text.strip().replace('$', '').replace(',', '')
    value = float(value_str)

    
    # Quit the driver
    driver.quit()
    
    return price, value


In [ ]:
get_set_cost('75292')

In [ ]:
set_money = {}
# looks like "set no : [price, value]"
for set_id in set_ids:
    try: set_money[set_id[0]] = get_set_cost(set_id[0])
    except: print("error with set_id: ", set_id[0])




In [ ]:
with open('set_money', 'w') as f:
    f.write(json.dumps(set_money))

In [ ]:
!curl -d "done" ntfy.sh/lego

In [4]:

import requests
import re

import requests
from bs4 import BeautifulSoup
import re
import json
from concurrent.futures import ThreadPoolExecutor

import concurrent.futures


def get_set_cost(set_num):
    # URL for set page on Brickeconomy
    url = f"https://www.brickeconomy.com/set/{set_num}-1/"
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"
    }
    # Send a GET request to the URL and get the HTML content
    try:
        response = requests.get(url, headers=headers)
        html_content = response.text
    except:
        return None, None

    # Extract the price and value using regex
    price_pattern = re.compile(r'<div class="row rowlist"><div class="col-xs-5 text-muted">Retail price</div><div class="col-xs-7">\$(.*?)</div></div>')
    value_pattern = re.compile(r'<div class="row rowlist"><div class="col-xs-5 text-muted">Value</div><div class="col-xs-7"><b>\$(.*?)</b></div></div>')

    # Search for the patterns in the HTML content
    price_match = price_pattern.search(html_content)
    value_match = value_pattern.search(html_content)

    # If no matches were found, return None for both price and value
    if not price_match or not value_match:
        return None, None

    # Extract the values from the matches
    price_str = price_match.group(1).replace(',', '')
    value_str = value_match.group(1).replace(',', '')

    # Convert the strings to floats and return
    price = float(price_str)
    value = float(value_str)
    
    # Write the results to a JSON file
    with open('set_prices.json', 'a') as f:
        data = {set_num: {'price': price, 'value': value}}
        json.dump(data, f)
        f.write('\n')

    return price, value

import tqdm
def get_set_costs_parallel(set_nums, max_workers=25):
    # Create a thread pool with the specified number of workers
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit a new job to the thread pool for each set number
        futures = [executor.submit(get_set_cost, set_num) for set_num in set_nums]
        # Wait for all jobs to complete and return the results
        return [future.result() for future in tqdm.tqdm(futures)]


In [5]:

set_nums = [set_id[0] for set_id in set_ids]
get_set_costs_parallel(set_nums)

100%|██████████| 4391/4391 [29:50<00:00,  2.45it/s]  


[(39.99, 398.22),
 (29.99, 103.4),
 (None, None),
 (29.99, 105.39),
 (12.99, 101.95),
 (19.99, 147.02),
 (14.99, 156.88),
 (19.99, 45.26),
 (12.99, 171.26),
 (7.99, 53.14),
 (4.99, 30.5),
 (7.99, 107.1),
 (4.99, 27.54),
 (1.99, 48.16),
 (2.99, 11.8),
 (1.99, 32.83),
 (None, None),
 (9.99, 49.35),
 (39.99, 215.81),
 (24.99, 149.29),
 (14.99, 72.63),
 (9.99, 41.15),
 (9.99, 25.06),
 (2.99, 44.98),
 (149.99, 309.9),
 (199.99, 871.46),
 (99.99, 377.72),
 (79.99, 196.0),
 (99.99, 212.89),
 (149.99, 256.43),
 (119.99, 141.66),
 (119.99, 518.0),
 (99.99, 559.79),
 (69.99, 497.2),
 (149.99, 459.76),
 (99.99, 476.92),
 (119.99, 304.76),
 (54.99, 327.82),
 (249.99, 2564.58),
 (149.99, 682.84),
 (99.99, 755.2),
 (None, None),
 (299.99, 1017.79),
 (149.99, 1787.43),
 (119.99, 820.36),
 (199.99, 1623.09),
 (139.99, 3179.22),
 (79.99, 752.24),
 (74.99, 347.75),
 (59.99, 505.28),
 (49.99, 514.72),
 (99.0, 1961.85),
 (49.99, 267.17),
 (199.0, 3735.75),
 (139.99, 1786.47),
 (89.99, 1153.32),
 (None, No

In [2]:
get_set_cost('75292')

None


(None, None)

In [6]:
set_ids

[('7075', "Captain Redbeard's Pirate Ship"),
 ('4860', "Doc Ock's Cafe Attack"),
 ('65462', 'Value Pack'),
 ('7074', 'Skull Island'),
 ('4668', 'Outrigger Construction Crane'),
 ('7073', 'Pirate Dock'),
 ('4858', "Doc Ock's Crime Spree"),
 ('4669', 'Turbo-Charged Police Boat'),
 ('7072', "Captain Kragg's Pirate Boat"),
 ('4667', "Loadin' Digger"),
 ('4666', 'Speedy Police Car'),
 ('7071', 'Treasure Island'),
 ('7070', 'Catapult Raft'),
 ('7081', 'Harry Hardtack and Monkey'),
 ('7080', 'Scurvy Dog and Crocodile'),
 ('7082', 'Cannonball Jimmy and Shark'),
 ('7290', 'Captain Kragg in Barrel'),
 ('7099', 'Accessory Motor'),
 ('4657', 'Fire Squad HQ'),
 ('4655', 'Quick Fix Station'),
 ('4654', 'Tanker Truck'),
 ('4653', 'Dump Truck'),
 ('4652', 'Tow Truck'),
 ('4651', 'Police Motorcycle'),
 ('10241', 'Maersk Line Triple-E'),
 ('10224', 'Town Hall'),
 ('10229', 'Winter Village Cottage'),
 ('10230', 'Mini Modulars'),
 ('10226', 'Sopwith Camel'),
 ('10218', 'Pet Shop'),
 ('10220', 'Volkswagen 